In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(os.path.dirname(os.getcwd()))

import json
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

from src.helper import lable_dict, scaler

In [2]:
if tf.config.list_physical_devices('GPU'):
    print("TensorFlow is using GPU")
else:
    print("TensorFlow is using CPU")

TensorFlow is using CPU


In [3]:
PROCESSED_DATA_DIR = "../data/processed"
TRAINING_DATA_DIR = "../data/split/train"
VALIDATION_DATA_DIR = "../data/split/val"
TEST_DATA_DIR = "../data/split/test"

BATCH_SIZE = 5

print(scaler)

MinMaxScaler()


In [4]:
def get_data_size(data):
    data_size = 0
    for dir in os.listdir(data):
        for _ in os.listdir(os.path.join(data, dir)):
            data_size += 1
    return data_size

print(get_data_size(TRAINING_DATA_DIR))
print(get_data_size(VALIDATION_DATA_DIR))

200
56


In [5]:
def data_generator(data_dir):
    """ 
    Generator function that yields data and labels from the given data directory.

    Args:
        data_dir (str): The directory containing the data.
    
    Yields:
        tuple: A tuple containing the data and the corresponding label. (data, label)
        data (list): A data sample.
        label (int): The label corresponding to the data sample.
    """
    subdirs = [os.path.join(data_dir, d) for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    label_to_index, _ = lable_dict(data_dir)
    for subdir in subdirs:
        data_files = [os.path.join(subdir, f) for f in os.listdir(subdir) if f.endswith('.json')]
        
        for data_file in data_files:
            with open(data_file, 'r') as f:
                data = json.load(f)

            data = [value for coordinate_dict in data for value in coordinate_dict.values()] 
            data = scaler.transform(np.array(data).reshape(-1, 1))
            data = data.flatten().tolist()
            label_index = label_to_index[os.path.basename(os.path.dirname(data_file))]
            yield (data, tf.constant(label_index, dtype=tf.int32))

In [6]:
training_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(TRAINING_DATA_DIR),
    output_signature=(
        tf.TensorSpec(shape=(63,), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)

validation_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(VALIDATION_DATA_DIR),
    output_signature=(
        tf.TensorSpec(shape=(63,), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)

test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(TEST_DATA_DIR),
    output_signature=(
        tf.TensorSpec(shape=(63,), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)

In [7]:
training_dataset = training_dataset.shuffle(1000).batch(BATCH_SIZE)
validation_dataset = validation_dataset.shuffle(1000).batch(BATCH_SIZE)
test_dataset = test_dataset.shuffle(1000).batch(BATCH_SIZE)

In [8]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(63,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [9]:
def train_model(model):
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,
                                                        restore_best_weights=True, min_delta= 0.01)

        model.fit(training_dataset.repeat(),
                steps_per_epoch=int(5* (get_data_size(TRAINING_DATA_DIR) / BATCH_SIZE)),
                validation_data=validation_dataset.repeat(),
                validation_steps=int(5* (get_data_size(VALIDATION_DATA_DIR) / BATCH_SIZE)),
                epochs=100,
                callbacks=[early_stopping]
        ) 

        return model

In [10]:
model = train_model(build_model())

loss, accuracy = model.evaluate(validation_dataset)
print(f"Test accuracy: {accuracy}")
print(f"Test loss: {loss}")

Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7014 - loss: 0.6125 - val_accuracy: 0.7311 - val_loss: 0.5182
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7847 - loss: 0.4960 - val_accuracy: 0.8077 - val_loss: 0.4374
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7585 - loss: 0.4818 - val_accuracy: 0.8385 - val_loss: 0.3997
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8216 - loss: 0.4230 - val_accuracy: 0.8598 - val_loss: 0.3397
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8583 - loss: 0.3352 - val_accuracy: 0.9346 - val_loss: 0.2871
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8722 - loss: 0.3010 - val_accuracy: 0.7615 - val_loss: 0.6053
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8758 - loss: 0.3205 - val_accuracy: 0.9129 - val_loss: 0.2840
Epoch 8/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8874 - loss: 0.2667 - val_accu

C:\Users\imani\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


In [11]:
choice = input("Do you want to save the model? (y/n): ")

if choice.lower() == 'y':
    model.save(f"../models/model_acc_{accuracy:.2f}_loss_{loss:.2f}.h5")
    print("Model saved successfully!")

Model saved successfully!
